In [0]:
!pip install transformers

     |████████████████████████████████| 481kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 29.1MB/s 
     |████████████████████████████████| 870kB 44.4MB/s 
     |████████████████████████████████| 3.1MB 38.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=3f8549be30e21c75144217ef9ec607f985c5b6d7c4ceef5ec75a3a9213264e33
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [2]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import  tqdm_notebook

Using TensorFlow backend.


In [0]:
directory_path = '/content/drive/My Drive/nlp'

In [0]:
hotel_df = pd.read_csv(os.path.join(directory_path,'hotel.csv'))
jokes_df = pd.read_csv(os.path.join(directory_path,'jokes.csv'))

In [60]:
hotel_df.shape


(10000, 25)

In [0]:
hotel_df['REVIEW'] = hotel_df['reviews.text']

In [0]:
hotel_df['REVIEW'] = hotel_df.REVIEW.str.lower()

In [0]:
hotel_df['REVIEW'] = hotel_df['REVIEW'].map(lambda k: '[REVIEW] '+str(k))

In [0]:
changed_text=hotel_df.REVIEW.apply(lambda x:x+"\n"+"\n")

In [67]:
open(os.path.join(directory_path,'data_lm_hotel.txt'), "w").write(''.join(changed_text))

3355809

In [69]:
#%cd /content/drive/My Drive/Bert Training/pytorch-transformers/examples/lm_finetuning
import os
os.listdir(os.getcwd())

['README.md',
 'simple_lm_finetuning.py',
 'pregenerate_training_data.py',
 'finetune_on_pregenerated.py',
 'data_lm_hotel.txt']

In [70]:
!python3 pregenerate_training_data.py --train_corpus data_lm_hotel.txt --bert_model bert-base-uncased --do_lower_case --output_dir training/ --epochs_to_generate 2 --max_seq_len 128

Loading Dataset: 20000 lines [00:09, 2209.52 lines/s]
Epoch:   0% 0/2 [00:00<?, ?it/s]
Document:   0% 0/10000 [00:00<?, ?it/s]
Document:   5% 505/10000 [00:00<00:01, 5043.48it/s]
Document:  10% 1047/10000 [00:00<00:01, 5148.80it/s]
Document:  15% 1473/10000 [00:00<00:01, 4844.25it/s]
Document:  19% 1866/10000 [00:00<00:01, 4516.85it/s]
Document:  23% 2331/10000 [00:00<00:01, 4555.82it/s]
Document:  28% 2797/10000 [00:00<00:01, 4585.36it/s]
Document:  33% 3315/10000 [00:00<00:01, 4748.24it/s]
Document:  38% 3850/10000 [00:00<00:01, 4912.71it/s]
Document:  43% 4341/10000 [00:00<00:01, 4910.91it/s]
Document:  48% 4815/10000 [00:01<00:01, 4821.34it/s]
Document:  54% 5352/10000 [00:01<00:00, 4973.76it/s]
Document:  58% 5849/10000 [00:01<00:00, 4972.47it/s]
Document:  63% 6342/10000 [00:01<00:00, 4957.64it/s]
Document:  68% 6844/10000 [00:01<00:00, 4973.98it/s]
Document:  73% 7345/10000 [00:01<00:00, 4983.13it/s]
Document:  79% 7859/10000 [00:01<00:00, 5027.37it/s]
Document:  84% 8361/10000 

In [71]:
os.listdir(os.getcwd())

['README.md',
 'simple_lm_finetuning.py',
 'pregenerate_training_data.py',
 'finetune_on_pregenerated.py',
 'data_lm_hotel.txt',
 'training']

In [72]:
!python3 finetune_on_pregenerated.py --pregenerated_data training/ --bert_model bert-base-uncased --do_lower_case --train_batch_size 16  --output_dir finetuned_lm/ --epochs 2

2020-02-16 08:06:38,693: device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
2020-02-16 08:06:38,856: loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2020-02-16 08:06:39,021: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
2020-02-16 08:06:39,021: Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size